#### Initial setup

In [1]:
# import packages

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv 
from lxml import etree
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

print("Finish importing packages")

Finish importing packages


In [2]:
# provide chromedriver path

PATH = "C:\Webdrivers\Chromedriver 69\chromedriver.exe"


# Open Chrome and Access Linkedin login site

driver = webdriver.Chrome(PATH)
sleep(3)

print('Web driver is launched successfully')

<ipython-input-2-08d7f396fa33>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Web driver is launched successfully


#### Extracting the data

In [3]:
# creating an empty list to store the data

Job_Title = []
Other = []  
    
# creating pagination

for page in range(1,6):
    
    url = f'https://jobs.jnj.com/en/jobs/?page={page}&pagesize=20#results'
    driver.get(url)
    driver.maximize_window()
    sleep(10)
    
    for title in driver.find_elements_by_xpath("//a[@class='stretched-link js-view-job']"):
        jobtitle = title.text
        Job_Title.append(jobtitle)
    
    sleep(2)
             
    for other in driver.find_elements_by_xpath('//ul[@class="list-inline job-meta"]'):
        other_details = other.text
        Other.append(other_details)
    
            
    print(f'Page {page} is scraped')
    
    sleep(5)
    
    

<ipython-input-3-034390644dea>:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for title in driver.find_elements_by_xpath("//a[@class='stretched-link js-view-job']"):
<ipython-input-3-034390644dea>:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for other in driver.find_elements_by_xpath('//ul[@class="list-inline job-meta"]'):


Page 1 is scraped
Page 2 is scraped
Page 3 is scraped
Page 4 is scraped
Page 5 is scraped


In [4]:
# convert the dictionary to dataframe

data = pd.DataFrame({
    
    'Job_Title' : Job_Title,
    'Other_details' : Other
})

#### basic dataframe overview

In [5]:
# check first five rows

data.head()

,Job_Title,Other_details
0,"Senior Finance Analyst, APAC Vision Deliver",Finance - Finance Singapore
1,Senior Therapeutic Specialist (Neuroscience) [...,"Sales - Selling Pharmaceutical Mongkok, China"
2,Key Account Manager [Janssen],"Sales - Selling Pharmaceutical Mongkok, China"
3,EMR,"Sales - Selling Pharmaceutical Tianjin, China"
4,Clinical Sales Specialist (Joint Reconstructio...,"Sales - Selling MD&D Richmond, United States"


In [6]:
# check the size of the data

data.shape

(100, 2)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Job_Title      100 non-null    object
 1   Other_details  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


#### data cleaning

In [8]:
# lets explore other_deatils 

data['Other_details']

0                           Finance - Finance Singapore
1         Sales - Selling Pharmaceutical Mongkok, China
2         Sales - Selling Pharmaceutical Mongkok, China
3         Sales - Selling Pharmaceutical Tianjin, China
4          Sales - Selling MD&D Richmond, United States
                            ...                        
95    Engineering - Process Engineering Leiden, Neth...
96    Info Technology - Business Solutions Chiyoda, ...
97         Sales - Selling MD&D Amersfoort, Netherlands
98         Operations - Procurement Manila, Philippines
99              Sales - Selling MD&D Kao-hsiung, Taiwan
Name: Other_details, Length: 100, dtype: object

In [9]:
# department and location needs to be separated into different columns

data['Other_details'][0]

'Finance - Finance Singapore'

In [10]:
# applyinh first split for separating all locations

data['Other_details'].str.split(';', expand=True)

,0,1,2,3,4
0,Finance - Finance Singapore,None,None,None,None
1,"Sales - Selling Pharmaceutical Mongkok, China",None,None,None,None
2,"Sales - Selling Pharmaceutical Mongkok, China",None,None,None,None
3,"Sales - Selling Pharmaceutical Tianjin, China",None,None,None,None
4,"Sales - Selling MD&D Richmond, United States",None,None,None,None
...,...,...,...,...,...
95,"Engineering - Process Engineering Leiden, Neth...",None,None,None,None
96,"Info Technology - Business Solutions Chiyoda, ...",None,None,None,None
97,"Sales - Selling MD&D Amersfoort, Netherlands",None,None,None,None
98,"Operations - Procurement Manila, Philippines",None,None,None,None


In [11]:
# applying second split for separating country

data['Other_details'].str.split(';', expand=True)[0].str.split(',', expand=True)

,0,1
0,Finance - Finance Singapore,None
1,Sales - Selling Pharmaceutical Mongkok,China
2,Sales - Selling Pharmaceutical Mongkok,China
3,Sales - Selling Pharmaceutical Tianjin,China
4,Sales - Selling MD&D Richmond,United States
...,...,...
95,Engineering - Process Engineering Leiden,Netherlands
96,Info Technology - Business Solutions Chiyoda,Japan
97,Sales - Selling MD&D Amersfoort,Netherlands
98,Operations - Procurement Manila,Philippines


In [12]:
# applying third split for separating cities

data['Other_details'].str.split(';', expand=True)[0].str.split(',', expand=True)[0].str.rpartition(' ')[[0,2]]

,0,2
0,Finance - Finance,Singapore
1,Sales - Selling Pharmaceutical,Mongkok
2,Sales - Selling Pharmaceutical,Mongkok
3,Sales - Selling Pharmaceutical,Tianjin
4,Sales - Selling MD&D,Richmond
...,...,...
95,Engineering - Process Engineering,Leiden
96,Info Technology - Business Solutions,Chiyoda
97,Sales - Selling MD&D,Amersfoort
98,Operations - Procurement,Manila


In [13]:
# allocating the splits into respective columns

data['Department'] = data['Other_details'].str.split(';', expand=True)[0].str.split(',', expand=True)[0].str.rpartition(' ')[0]
data['City'] = data['Other_details'].str.split(';', expand=True)[0].str.split(',', expand=True)[0].str.rpartition(' ')[2]
data['Country'] = data['Other_details'].str.split(';', expand=True)[0].str.split(',', expand=True)[1]

In [14]:
# repeating split for remaing columns to get all locations

data['Location_2'] = data['Other_details'].str.split(';', expand=True)[1]
data['Location_3'] = data['Other_details'].str.split(';', expand=True)[2]
data['Location_4'] = data['Other_details'].str.split(';', expand=True)[3]
data['Location_5'] = data['Other_details'].str.split(';', expand=True)[4]

In [15]:
# dropping unwanted columns

data.drop('Other_details', axis=1, inplace=True)

In [24]:
# combining city and country to create location 1 column

data['Location_1'] = (data['City'] + ',' + data['Country'].fillna('')).str.rstrip(',')

In [27]:
# creating final dataset

data = data[['Job_Title', 'Department', 'Location_1', 'Location_2', 'Location_3', 'Location_4', 'Location_5']]

In [28]:
# preview final prepared dataset

data.head(10)

,Job_Title,Department,Location_1,Location_2,Location_3,Location_4,Location_5
0,"Senior Finance Analyst, APAC Vision Deliver",Finance - Finance,Singapore,None,None,None,None
1,Senior Therapeutic Specialist (Neuroscience) [...,Sales - Selling Pharmaceutical,"Mongkok, China",None,None,None,None
2,Key Account Manager [Janssen],Sales - Selling Pharmaceutical,"Mongkok, China",None,None,None,None
3,EMR,Sales - Selling Pharmaceutical,"Tianjin, China",None,None,None,None
4,Clinical Sales Specialist (Joint Reconstructio...,Sales - Selling MD&D,"Richmond, United States",None,None,None,None
5,Key Account Manager,Sales - Selling MD&D,"Lucknow, India",None,None,None,None
6,Process Excellence Lead,Engineering - Process Engineering,"Groningen, Netherlands",None,None,None,None
7,ABAP Developer,Info Technology - Development,"Shanghai, China",None,None,None,None
8,MR,Sales - Selling Pharmaceutical,"Haikou, China",None,None,None,None
9,District Manager- Lucknow,Sales - Selling Pharmaceutical,"Lucknow, India",None,None,None,None


In [ ]:
# dumping it into excel file

data.to_excel('output.xlsx')

In [17]:
# aborting selenium operation by quitting browser

driver.quit()